In [1]:
import torch
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import gc
import os


# 시각화 관련 설정
try:
    plt.rcParams['font.family'] = 'Apple SD Gothic Neo'
except:
    try:
        plt.rcParams['font.family'] = 'NanumGothic'
    except:
        plt.rcParams['font.family'] = 'AppleGothic'

plt.rcParams['axes.unicode_minus'] = False
fm._load_fontmanager(try_read_cache=False)


# 디바이스 설정
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    DEVICE = torch.device("mps") # 맥 GPU
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda:0") # 윈도우 GPU
else:
    DEVICE = torch.device("cpu") # CPU


# 캐시 지우기 함수 생성
def clean_cache():
    gc.collect()
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()

# MallocStackLogging 에러 출력 방지
os.environ.pop("MallocStackLogging", None)
os.environ.pop("MallocStackLoggingNoCompact", None)
os.environ.pop("DYLD_INSERT_LIBRARIES", None)


# # 로그
# import logging

# def init_logger() -> logging.Logger:
#     logging.basicConfig(
#         format="%(asctime)s [%(levelname)s] (%(filename)s:%(lineno)d) - %(message)s",
#         datefmt="%Y-%m-%d %H:%M:%S",
#         level=logging.INFO,
#         encoding="utf-8",
#     )
#     return logging.getLogger("")

# logger = init_logger()

Matplotlib is building the font cache; this may take a moment.


In [ ]:
import json

ROOT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(ROOT_DIR, "data")
IMAGE_DIR = os.path.join(DATA_DIR, "train_images")
ANNOT_DIR = os.path.join(DATA_DIR, "train_annotations")

full_dict_path = os.path.join(DATA_DIR, "FULL_DICT.json")
err_txt_path = os.path.join(DATA_DIR, "err_image_paths.txt")
fixed_dict_path = os.path.join(DATA_DIR, "FIXED_DICT.json")

FINAL_DICT = {}

# FULL_DICT 더하기
try:
    with open(full_dict_path, "r", encoding="utf-8") as f:
        FULL_DICT = json.load(f)
except:
    pass

for key, value in FULL_DICT.items():
    a = os.path.join(IMAGE_DIR, key)
    tmp_list = []
    for pa in value:
        tmp_list.append(os.path.join(ANNOT_DIR, pa))
    FINAL_DICT[a] = tmp_list


# err_image_paths 빼기
try:
    with open(err_txt_path, "r", encoding="utf-8") as f:
        err_image_paths = f.read().split()
    for err in err_image_paths:
        a = os.path.join(IMAGE_DIR, err)
        del FINAL_DICT[a]
except:
    pass


# FIXED_DICT 더하기
try:
    with open(fixed_dict_path, "r", encoding="utf-8") as f:
        FIXED_DICT = json.load(f)
    for key, value in FIXED_DICT:
        a = os.path.join(IMAGE_DIR, key)
        tmp_list = []
        for pa in value:
            tmp_list.append(os.path.join(ANNOT_DIR, pa))
        FINAL_DICT[a] = tmp_list
except:
    pass


for image_path, annot_paths in FINAL_DICT.items():

    tmp_list = []

    for path in annot_paths:
        bbox = []
        with open(path, "r", encoding="utf-8") as f:
            json_data = json.load(f)

        xywh_bbox = json_data["annotations"][0]["bbox"]

        tmp_list.append({"bbox": xywh_bbox,
                        "label": json_data["categories"][0]["id"]})
        
    FINAL_DICT[image_path] = tmp_list

FINAL_DICT

In [ ]:
# 기타 증강 등등 여기서